In [18]:
import pandas as pd
import string
import re
from collections import Counter

f = open('data', 'wb')
df_test = pd.read_csv('test.csv', sep=',')
pat = re.compile(f'[{string.punctuation}]')
df_test = df_test.replace(pat, '')

df_test['question1'] = df_test['question1'].str.lower()
df_test['question1'] = df_test['question1'].str.split()
df_test['question2'] = df_test['question2'].str.lower()
df_test['question2'] = df_test['question2'].str.split()
del pat

In [22]:
#лемматизация
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sasha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sasha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))

def stop_words(text):
    if type(text) is not list:
        return []
    return list(set(text) - en_stops)

df_test['q1_lemmatized'] = df_test['question1'].apply(stop_words)
df_test['q2_lemmatized'] = df_test['question2'].apply(stop_words)

In [1]:
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from tqdm.notebook import tqdm_notebook

tqdm_notebook.pandas()
tag_dict = {"J": wordnet.ADJ,
            "N": wordnet.NOUN,
            "V": wordnet.VERB,
            "R": wordnet.ADV}

lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    if type(text) is not list:
        return []
    tags = list(map(lambda x: tag_dict.get(x[1][0].upper(), wordnet.NOUN), nltk.pos_tag(text)))
    return [lemmatizer.lemmatize(w, t) for w, t in zip(text, tags)]


df_test['q1_lemmatized'] = df_test['q1_lemmatized'].apply(lemmatize_text)
df_test['q2_lemmatized'] = df_test['q2_lemmatized'].apply(lemmatize_text)

NameError: name 'nltk' is not defined

In [1]:
import pickle

# pickle.dump(df_test, open("lem_df", "wb"))
df_test = pickle.load(open("dumped", "rb"))

In [2]:
from collections import Counter
#для каждого слова из 1 вопроса считает количество документов, которые его сожержат
import numpy as np

def func(x) :
    if type(x) is list :
        list(set(x)) 
    else:
        list()

list1 = np.hstack(np.array(list(map(func, df_test["q1_lemmatized"].tolist()))))
q1 = Counter(list1)
del list1

In [3]:
#для каждого слова из 2 вопроса считает количество документов, которые его сожержат
list2 = np.hstack(np.array(list(map(lambda x: list(set(x)) if type(x) is list else list(), df_test["q2_lemmatized"].tolist()))))
q2 = Counter(list2)
del list2

In [4]:
q = q1 + q2
del q1
del q2

In [5]:
#считаем tf
import numpy as np
def tf(val):
    if type(val) is list:
        q_keys =  Counter(val).keys()
        q_vals = np.array(list(Counter(val).values()))
        q_vals = q_vals / len(val)
        return dict(zip(q_keys, q_vals))
    else:
        return dict()
    
df_test["tf1"] = df_test["q1_lemmatized"].apply(tf)
df_test["tf2"] = df_test["q2_lemmatized"].apply(tf)


In [6]:
# считаем idf
q_keys = q.keys()
q_vals = np.array(list(q.values()))
q_vals = np.log(len(df_test) / q_vals)
idf = dict(zip(q_keys, q_vals))
del q_keys
del q_vals

In [7]:
#считаем tf-idf

def mul_idf(val):
    for word in val:
        val[word] *= idf[word]
    return val

df_test["tf-idf1"] = df_test["tf1"].apply(mul_idf)
df_test["tf-idf2"] = df_test["tf2"].apply(mul_idf)


In [8]:
df_test["words_1"] = df_test["tf-idf1"].apply(lambda x: set(x.keys()))
df_test["words_2"] = df_test["tf-idf2"].apply(lambda x: set(x.keys()))


In [9]:
#делаем общий вектор из слов двух предложений
def or_func(x):
    return list(x["words_1"] | x["words_2"])

df_test["all_words"] = df_test.apply(or_func, axis=1)

In [10]:
#создаем два вектора и считаем косинус между ними
from scipy.spatial import distance
from tqdm.notebook import tqdm_notebook

tqdm_notebook.pandas()

def cosine(x):
    l = len(x["all_words"])
    a1 = [0] * l
    a2 = [0] * l
    for i in range(l):
        if x["all_words"][i] in x["tf-idf1"]:
            a1[i] = x["tf-idf1"][x["all_words"][i]]
        if x["all_words"][i] in x["tf-idf2"]:
        
            a2[i] = x["tf-idf2"][x["all_words"][i]]
    return distance.cosine(a1, a2)

df_test["cosine"] = df_test.progress_apply(cosine, axis=1)

c:\users\sasha\pycharmprojects\quora\venv\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
c:\users\sasha\pycharmprojects\quora\venv\lib\site-packages\scipy\spatial\distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
c:\users\sasha\pycharmprojects\quora\venv\lib\site-packages\numpy\lib\function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
c:\users\sasha\pycharmprojects\quora\venv\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [11]:

df_test["cosine"] = df_test["cosine"].fillna(1)
df_test["cosine"] = df_test["cosine"].apply(lambda x: 1 - x)

In [12]:
import pandas as pd
#записываем результат в файл simple_xgb.csv
sub = pd.DataFrame()
sub['test_id'] = df_test['test_id']
sub['is_duplicate'] = df_test['cosine']
sub.to_csv('simple_xgb.csv', index=False)